# Projeto 01: Análise dos Aluguéis do Rio de Janeiro com Python
Usando Python para analisar os dados de um dataset fictício dos alugueis do Rio de Janeiro

## Objetivos: 
Este dataset faz parte do treinamento de Pandas da Alura e o objetivo desse projeto é expandir o conhecimento adquirido no treinamento para além das funções básicas, utilizando outros recursos e outras formas de explorar esse dataset e promover uma análise sobre os dados.

## Extraindo e Explorando o dataset

In [1]:
# Registrando a versão do Python
versao =!python -V
print('Versão: ', versao)

Versão:  ['Python 3.8.8']


In [2]:
# Importando bibliotecas
import pandas as pd
# inline mostrara um gráfico estática na tela
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
# Importando dataset
df = pd.read_csv('dataset/aluguel.csv', sep=';')

#### Dicionário das variáveis

###### <b>Tipo :</b>Tipo do imóvel
###### <b>Bairro: </b>Bairro onde está localizado
###### <b>Quartos: </b>Quantidade de quartos do imóvel
###### <b>Vagas: </b>Quantidade de vagas do imóvel
###### <b>Suítes: </b>Quantidade de suítes do ímovel<
###### <b>Área: </b>Área do imóvel
###### <b>Valor: </b>Valor do Aluguel do imóvel
###### <b>Condomínio: </b>Valor do condomínio dos imóveis que possuem
###### <b>IPTU: </b>Valor do IPTU do imóvel
###### <b>Valor por m²: </b>Valor do m² do imóvel. Variável será criada 



In [4]:
# Visualizando os 5 primeiros registros do dataset
df.head(5)

,Tipo,Bairro,Quartos,Vagas,Suites,Area,Valor,Condominio,IPTU
0,Quitinete,Copacabana,1,0,0,40,1700.0,500.0,60.0
1,Casa,Jardim Botânico,2,0,1,100,7000.0,NaN,NaN
2,Conjunto Comercial/Sala,Barra da Tijuca,0,4,0,150,5200.0,4020.0,1111.0
3,Apartamento,Centro,1,0,0,15,800.0,390.0,20.0
4,Apartamento,Higienópolis,1,0,0,48,800.0,230.0,NaN


In [5]:
# Número de linhas e colunas inciais
df.shape

(32960, 9)

## Limpeza e Transformação

In [6]:
# Inserindo um nome para a coluna 
df.columns.name='ID'
df.head(5)

ID,Tipo,Bairro,Quartos,Vagas,Suites,Area,Valor,Condominio,IPTU
0,Quitinete,Copacabana,1,0,0,40,1700.0,500.0,60.0
1,Casa,Jardim Botânico,2,0,1,100,7000.0,NaN,NaN
2,Conjunto Comercial/Sala,Barra da Tijuca,0,4,0,150,5200.0,4020.0,1111.0
3,Apartamento,Centro,1,0,0,15,800.0,390.0,20.0
4,Apartamento,Higienópolis,1,0,0,48,800.0,230.0,NaN


In [7]:
# Informações dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32960 entries, 0 to 32959
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Tipo        32960 non-null  object 
 1   Bairro      32960 non-null  object 
 2   Quartos     32960 non-null  int64  
 3   Vagas       32960 non-null  int64  
 4   Suites      32960 non-null  int64  
 5   Area        32960 non-null  int64  
 6   Valor       32943 non-null  float64
 7   Condominio  28867 non-null  float64
 8   IPTU        22723 non-null  float64
dtypes: float64(3), int64(4), object(2)
memory usage: 2.3+ MB


É possível verificar que as variáveis Valor, Condomínio e IPTU tem dados nulos

In [8]:
# Definindo uma função para visualizar o total de nulos e seu percentual
def nulos(df):
    total = df.isnull().sum() 
    percent = df.isnull().sum()/df.isnull().count()
    missing_data = pd.concat([total, percent], axis=1, sort=False, keys=['total', 'percent'])
    missing_data[missing_data['percent']!=0]
    return missing_data

In [9]:
nulos(df)

,total,percent
ID,,
Tipo,0,0.000000
Bairro,0,0.000000
Quartos,0,0.000000
Vagas,0,0.000000
Suites,0,0.000000
Area,0,0.000000
Valor,17,0.000516
Condominio,4093,0.124181
IPTU,10237,0.310589


### Tratando a variável VALOR

Para está variável será realizada a exclusão desses registros com dados nulos

In [10]:
df[df['Valor'].isnull()].shape[0]

17

In [11]:
# Removendo missing values
A = df.shape[0]
df.dropna(subset = ['Valor'], inplace = True)
B = df.shape[0]
A - B

17

In [12]:
# Verificando se ainda há registro nulo para valor
df[df['Valor'].isnull()].shape[0]

0

### Tratando a variável CONDOMÍNIO

Para está variável será assumido que todo apartamento tem que ter valor de condomínio, portanto, os registros sem valor serão descartados

In [13]:
df[df['Condominio'].isnull()].shape[0]

4089

In [14]:
# Selecionando os registros dos apartamentos sem valor de condominio
selecao1 = (df['Tipo'] == 'Apartamento') & (df['Condominio'].isnull())

In [15]:
# Removendo registros
A = df.shape[0]
df = df[~selecao1] # colocamos o '~' na frente da seleção para que ele inverta a seleção 
B = df.shape[0]
A - B

745

In [16]:
# Verificando quantos registros nulos ficarão após a remoção
df[df['Condominio'].isnull()].shape[0]

3344

### Tratando as variáveis CONDOMÍNIO e IPTU

In [17]:
# Para os registros que permanecerem com valor nulo, vamos atribuir o valor 0
df = df.fillna({'Condominio': 0, 'IPTU': 0})
df

ID,Tipo,Bairro,Quartos,Vagas,Suites,Area,Valor,Condominio,IPTU
0,Quitinete,Copacabana,1,0,0,40,1700.0,500.0,60.0
1,Casa,Jardim Botânico,2,0,1,100,7000.0,0.0,0.0
2,Conjunto Comercial/Sala,Barra da Tijuca,0,4,0,150,5200.0,4020.0,1111.0
3,Apartamento,Centro,1,0,0,15,800.0,390.0,20.0
4,Apartamento,Higienópolis,1,0,0,48,800.0,230.0,0.0
...,...,...,...,...,...,...,...,...,...
32955,Quitinete,Centro,0,0,0,27,800.0,350.0,25.0
32956,Apartamento,Jacarepaguá,3,1,2,78,1800.0,800.0,40.0
32957,Apartamento,São Francisco Xavier,2,1,0,48,1400.0,509.0,37.0
32958,Apartamento,Leblon,2,0,0,70,3000.0,760.0,0.0


In [18]:
# Chamando a função de nulo para verificar se ainda há algum valor nulo no dataframe
nulos(df)

,total,percent
ID,,
Tipo,0,0.0
Bairro,0,0.0
Quartos,0,0.0
Vagas,0,0.0
Suites,0,0.0
Area,0,0.0
Valor,0,0.0
Condominio,0,0.0
IPTU,0,0.0


### Salvando uma nova versão do dataset com as devidas correções

In [36]:
# Exportando como arquivo CSV
df.to_csv('dataset\df_limpo_v1.csv', sep=';', index = False)

In [41]:
# Lendo novo df
df = pd.read_csv('dataset\df_limpo_v1.csv', sep=';')
df.head(10)

,Tipo,Bairro,Quartos,Vagas,Suites,Area,Valor,Condominio,IPTU
0,Quitinete,Copacabana,1,0,0,40,1700.0,500.0,60.0
1,Casa,Jardim Botânico,2,0,1,100,7000.0,0.0,0.0
2,Conjunto Comercial/Sala,Barra da Tijuca,0,4,0,150,5200.0,4020.0,1111.0
3,Apartamento,Centro,1,0,0,15,800.0,390.0,20.0
4,Apartamento,Higienópolis,1,0,0,48,800.0,230.0,0.0
5,Apartamento,Cachambi,2,0,0,50,1300.0,301.0,17.0
6,Casa de Condomínio,Barra da Tijuca,5,4,5,750,22000.0,0.0,0.0
7,Casa de Condomínio,Ramos,2,2,0,65,1000.0,0.0,0.0
8,Conjunto Comercial/Sala,Centro,0,3,0,695,35000.0,19193.0,3030.0
9,Apartamento,Grajaú,2,1,0,70,1500.0,642.0,74.0


In [46]:
tipos_imoveis = df.Tipo.drop_duplicates().to_list()
tipos_imoveis

['Quitinete',
 'Casa',
 'Conjunto Comercial/Sala',
 'Apartamento',
 'Casa de Condomínio',
 'Prédio Inteiro',
 'Flat',
 'Loja/Salão',
 'Galpão/Depósito/Armazém',
 'Casa Comercial',
 'Casa de Vila',
 'Terreno Padrão',
 'Box/Garagem',
 'Loft',
 'Loja Shopping/ Ct Comercial',
 'Chácara',
 'Loteamento/Condomínio',
 'Sítio',
 'Pousada/Chalé',
 'Studio',
 'Hotel',
 'Indústria']

In [47]:
len(tipos_imoveis)

22

Nota-se que há que há 22 tipos de imóveis entre residenciais e comerciais. A análise a ser realizada são somente dos imóveis residenciais, portanto, os comerciais podem ser descartados.

#### Selecionando imóveis residenciais

In [48]:
# Criando uma variável com uma lista somente de imóveis residenciais
residencial = ['Quitinete',
 'Casa',
 'Apartamento',
 'Casa de Condomínio',
 'Casa de Vila']

In [50]:
# O método isin vai procurar no dataframe os valores contidos em residencial e se achar ficam como verdadeiro, senão como falso
selecao = df['Tipo'].isin(residencial)

In [51]:
# Será passada a seleção dos imóveis para o dataframe
df_residencial = df[selecao]
df_residencial

,Tipo,Bairro,Quartos,Vagas,Suites,Area,Valor,Condominio,IPTU
0,Quitinete,Copacabana,1,0,0,40,1700.0,500.0,60.0
1,Casa,Jardim Botânico,2,0,1,100,7000.0,0.0,0.0
3,Apartamento,Centro,1,0,0,15,800.0,390.0,20.0
4,Apartamento,Higienópolis,1,0,0,48,800.0,230.0,0.0
5,Apartamento,Cachambi,2,0,0,50,1300.0,301.0,17.0
...,...,...,...,...,...,...,...,...,...
32191,Apartamento,Méier,2,0,0,70,900.0,490.0,48.0
32193,Quitinete,Centro,0,0,0,27,800.0,350.0,25.0
32194,Apartamento,Jacarepaguá,3,1,2,78,1800.0,800.0,40.0
32195,Apartamento,São Francisco Xavier,2,1,0,48,1400.0,509.0,37.0


Agora o dataframe está menor, mas contém apenas os dados que vamos analisar

## Análise após Tratamento

### Quais os tipos de Imóveis Residenciais?